In [8]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
import time

# 需要什么读取什么不要一次性全都读取了，影响程序速度

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

time1 = time.time()

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)
    

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)

    
# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()

# 提取B区信息
B_DF = DTBS_DataFrame['B']

In [9]:
# 获取到目前为止已退市可转债代码，这里的目的是希望在买入时规避掉这些即将退市的转债
delisted_bond_codes = list(B_DF.loc[B_DF['dld'] < '2046-01-01', 'bond_code'])
print(delisted_bond_codes)

['123057.SZ', '128014.SZ', '128079.SZ', '113570.SH', '113622.SH', '123014.SZ', '123134.SZ', '123136.SZ', '123148.SZ', '127036.SZ', '111006.SH', '113016.SH', '123075.SZ', '127004.SZ', '128100.SZ', '128114.SZ', '110057.SH', '113585.SH', '123029.SZ', '123098.SZ', '127057.SZ', '113011.SH', '113013.SH', '123015.SZ', '123077.SZ', '123083.SZ', '123116.SZ', '127021.SZ', '128078.SZ', '128140.SZ', '113567.SH', '123105.SZ', '128145.SZ', '128040.SZ', '123060.SZ', '123125.SZ', '113525.SH', '128029.SZ', '123123.SZ', '128022.SZ', '128069.SZ', '128015.SZ', '110056.SH', '123139.SZ', '123110.SZ', '113642.SH', '110038.SH', '128046.SZ', '128073.SZ', '123027.SZ', '113635.SH', '123137.SZ', '118001.SH', '113502.SH', '123023.SZ', '123062.SZ', '123097.SZ', '127048.SZ', '113620.SH', '123045.SZ', '123086.SZ', '123084.SZ', '128139.SZ', '123073.SZ', '113548.SH', '113630.SH', '123070.SZ', '113599.SH', '128013.SZ', '128107.SZ', '110071.SH', '127013.SZ', '113541.SH', '113568.SH', '127003.SZ', '113550.SH', '118002.SH'

In [10]:
day_list  = sorted(list(set(A_DF['date'])))

# 设定选择区间为20天，得到低价中点与低价程度两列，利用滑动平均实现
A_DF.sort_values(['bond_code', 'date'], inplace=True)
grouped = A_DF.groupby('bond_code')
n = 20

# 两种不同的计算方式，一种是计算前20天收盘价的最高价、最低价（不含当天），一种是计算前20天收盘价的最高价、最低价（含当天）
# A_DF['highest_price_n_days'] = grouped['dp'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].max(), raw=True).reset_index(level=0, drop=True)
# A_DF['lowest_price_n_days'] = grouped['dp'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].min(), raw=True).reset_index(level=0, drop=True)

A_DF['highest_price_n_days'] = grouped['dp'].rolling(window=n, min_periods=n).max().reset_index(level=0, drop=True)
A_DF['lowest_price_n_days'] = grouped['dp'].rolling(window=n, min_periods=n).min().reset_index(level=0, drop=True)


A_DF['low_price_level'] = A_DF['dp'] / A_DF['lowest_price_n_days'] - 1 
A_DF['low_price_midpoint'] = (A_DF['highest_price_n_days'] + A_DF['lowest_price_n_days']) / 2

A_DF['dp_median'] = A_DF.groupby('date')['dp'].transform('median')

print(A_DF)


# 设置测试起始日期与终止日期
start_day = '2020-01-02'
end_day = '2023-09-05'

start_index = day_list.index(start_day)
end_index = day_list.index(end_day)

day_list = day_list[start_index:end_index+1]


# 最后需要展示的列
need_columns = ['date' ,'bond_code' ,'dp','sn','change', 'dl','cpr','bl','yl']
df_select = pd.DataFrame(columns = need_columns)

# temp_DF是临时性的
temp_DF = pd.merge(A_DF,B_DF[['bond_code','sn', 'qt1']],on='bond_code',how='left')

# 直接剔除掉ST股，筛选出指定时间范围，应不应该剔除需要看具体情况来定
temp_DF = temp_DF[(temp_DF['date'] >= start_day) & 
                  (temp_DF['date'] <= end_day)  &
                  (~temp_DF['sn'].str.contains("ST"))
                 ].copy()


print('处理前的数据：\n', temp_DF)

temp_DF_delisted = temp_DF[temp_DF['bond_code'].isin(delisted_bond_codes)].copy()

# 按照'code'列进行分组，然后对'date'列的值进行排名
temp_DF_delisted['Rank'] = temp_DF_delisted.groupby('bond_code')['date'].rank(ascending=False)

# 剔除'Rank'列值小于等于Top N的行
top_n = 5
temp_DF_delisted = temp_DF_delisted[temp_DF_delisted['Rank'] > top_n].copy()

# 删除辅助列'Rank'
temp_DF_delisted.drop('Rank', axis=1, inplace=True)
print('已退市转债（去除最后5天的信息）：\n', temp_DF_delisted)

temp_DF_normal = temp_DF[~temp_DF['bond_code'].isin(delisted_bond_codes)].copy()

print('未退市转债（保留全部信息）：\n', temp_DF_normal)

temp_DF = pd.concat([temp_DF_delisted,temp_DF_normal],ignore_index=True)

print('处理后的数据：\n', temp_DF)



# 下面输出5个老策略

# 高ytm策略
high_ytm_5 = temp_DF[(temp_DF['dp'] >= 100) & 
                     (temp_DF['dp'] <= 130) & 
                     (temp_DF['ytm'] >= 0.5) & 
                     (temp_DF['bl'] > 0) & 
                     (temp_DF['bl'] <= 7) &
                     (temp_DF['trt'] >= 2) &
                     (temp_DF['trt'] <= 10)].copy()
high_ytm_5.sort_values(['date','ytm'],ascending=[True,False] ,inplace=True)
high_ytm_5 = high_ytm_5.groupby(['date']).head(5)
print('高ytm：\n', high_ytm_5)



# 小市值3双低5
small_market_value_double_low_5 = temp_DF[(temp_DF['dp'] >= 100)&
                                          (temp_DF['dp'] <= 130)&
                                          (temp_DF['bl'] > 0)&
                                          (temp_DF['bl'] <= 3)].copy()
small_market_value_double_low_5.sort_values(['date','dl'],ascending=[True,True] ,inplace=True)
small_market_value_double_low_5 = small_market_value_double_low_5.groupby(['date']).head(5)
print('小市值3双低5：\n', small_market_value_double_low_5)



# 双低5，double_low_5
double_low_5 = temp_DF[(temp_DF['dp'] >= 100) &
                       (temp_DF['dp'] <= 130) &
                       (temp_DF['ytm'] >= -1) &
                       (temp_DF['bl'] > 0) &
                       (temp_DF['bl'] <= 7) &
                       (temp_DF['trt'] >= 2)&
                       (temp_DF['trt'] <= 10)].copy()
double_low_5.sort_values(['date','dl'],ascending=[True,True] ,inplace=True)
double_low_5 = double_low_5.groupby(['date']).head(5)
print('双低5：\n', double_low_5)



# 区间双低3，range_double_low_3
range_double_low_3_zone1 = temp_DF[(temp_DF['dp'] >= 110) &
                                   (temp_DF['dp'] < 120) &
                                   (temp_DF['ytm'] >= -1) &
                                   (temp_DF['bl'] > 0) &
                                   (temp_DF['bl'] <= 7) &
                                   (temp_DF['trt'] >= 2) &
                                   (temp_DF['trt'] <= 10)].copy()
range_double_low_3_zone1.sort_values(['date','dl'],ascending=[True,True] ,inplace=True)
range_double_low_3_zone1 = range_double_low_3_zone1.groupby(['date']).head(3)

range_double_low_3_zone2 = temp_DF[(temp_DF['dp'] >= 120) &
                                   (temp_DF['dp'] <= 130) &
                                   (temp_DF['ytm'] >= -1) &
                                   (temp_DF['bl'] > 0) &
                                   (temp_DF['bl'] <= 7) &
                                   (temp_DF['trt'] >= 2) &
                                   (temp_DF['trt'] <= 10)].copy()
range_double_low_3_zone2.sort_values(['date','dl'],ascending=[True,True] ,inplace=True)
range_double_low_3_zone2 = range_double_low_3_zone2.groupby(['date']).head(3)

range_double_low_3 = pd.concat([range_double_low_3_zone1,range_double_low_3_zone2],axis=0)
print('区间双低3：\n', range_double_low_3)


#低价小市值，low_price_small_market_value_5
low_price_small_market_value_5 = temp_DF[(temp_DF['dp'] >= 110) & 
                                         (temp_DF['dp'] <= 130) & 
                                         (temp_DF['bl'] > 0) &
                                         (temp_DF['bl'] < 3) &
                                         # (temp_DF['ytm'] > -1) &
                                         (temp_DF['cpr'] < 50) &
                                         (temp_DF['dp'] < temp_DF['low_price_midpoint'])].copy()
low_price_small_market_value_5.sort_values(['date','low_price_level'],ascending=[True,True] ,inplace=True)
low_price_small_market_value_5 = low_price_small_market_value_5.groupby(['date']).head(5)
print('低价小市值5：\n', low_price_small_market_value_5)



# 后面修改的
# 低价策略，根据姚迁迁思路对"低价小市值"策略进行修改
low_price_5 = temp_DF[(temp_DF['dp'] >= temp_DF['dp_median']-8) & 
                      (temp_DF['dp'] <= temp_DF['dp_median']+3) & 
                      (temp_DF['bl'] > 0) &
                      (temp_DF['bl'] < 7) &
                      # (temp_DF['ytm'] > -1) &
                      (temp_DF['cpr'] < 50) &
                      (temp_DF['dp'] < temp_DF['low_price_midpoint'])].copy()
low_price_5.sort_values(['date','low_price_level'],ascending=[True,True] ,inplace=True)
low_price_5 = low_price_5.groupby(['date']).head(5)
print('低价5（基于低价小市值5策略修改得来）：\n', low_price_5)



# 双低+平底 From 20223-08-31
double_low_pd_5 = temp_DF[(temp_DF['dp'] >= 100) &
                       (temp_DF['dp'] <= 130) &
                       (temp_DF['ytm'] >= -1) &
                       (temp_DF['bl'] > 0) &
                       (temp_DF['bl'] <= 7) &
                       (temp_DF['trt'] >= 2)&
                       (temp_DF['trt'] <= 10)].copy()
double_low_pd_5['pd+dl'] = double_low_pd_5['pd'] + double_low_pd_5['dl']
double_low_pd_5.sort_values(['date','pd+dl'],ascending=[True,True] ,inplace=True)
double_low_pd_5 = double_low_pd_5.groupby(['date']).head(5)
print('平底+双低：\n', double_low_pd_5)






                      bond_code        date      cpr       dp        bl  \
id                                                                        
110030.SH,2017-01-03  110030.SH  2017-01-03  41.5266  114.040  9.780730   
110030.SH,2017-01-04  110030.SH  2017-01-04  40.3232  114.230  9.780730   
110030.SH,2017-01-05  110030.SH  2017-01-05  40.2584  114.950  9.780730   
110030.SH,2017-01-06  110030.SH  2017-01-06  41.3362  114.860  9.780730   
110030.SH,2017-01-09  110030.SH  2017-01-09  42.6442  115.530  9.780730   
...                         ...         ...      ...      ...       ...   
128145.SZ,2023-02-20  128145.SZ  2023-02-20  -0.7001  131.670  1.187342   
128145.SZ,2023-02-21  128145.SZ  2023-02-21  -0.3236  130.449  1.099123   
128145.SZ,2023-02-22  128145.SZ  2023-02-22  -0.5313  127.030  0.976149   
128145.SZ,2023-02-23  128145.SZ  2023-02-23  -0.2378  125.205  0.773509   
128145.SZ,2023-02-24  128145.SZ  2023-02-24  -0.3382  122.499  0.156706   

                        

         bond_code        date      cpr       dp       bl     trt     ytm  \
8570    110043.SH  2020-01-02  30.7512  111.700  29.9904  0.6241 -0.4722   
8571    110043.SH  2020-01-03  30.1885  110.820  29.9903  0.4232 -0.2759   
8572    110043.SH  2020-01-06  30.3055  110.320  29.9902  0.5990 -0.1636   
8573    110043.SH  2020-01-07  28.8169  109.850  29.9902  0.4257 -0.0570   
8574    110043.SH  2020-01-08  30.0650  108.720  29.9902  0.4460  0.2021   
...           ...         ...      ...      ...      ...     ...     ...   
395840  128144.SZ  2023-08-30  51.5100  116.087   9.7940  0.5100 -0.7000   
395841  128144.SZ  2023-08-31  50.7800  115.527   9.7940  0.2800 -0.5600   
395842  128144.SZ  2023-09-01  50.5000  116.000   9.7940  0.5100 -0.6800   
395843  128144.SZ  2023-09-04  47.0900  117.000   9.7940  0.5400 -0.9300   
395844  128144.SZ  2023-09-05  45.9600  117.299   9.7940  0.1600 -1.0000   

            yl        dl  ia  ...  xx30   hs  redeem_count  \
8570    4.0767  142.4512

低价小市值5：
         bond_code        date      cpr       dp       bl        trt      ytm  \
15232   113514.SH  2020-01-02   9.5646  110.920  0.96519   9.598110   0.0463   
15233   113514.SH  2020-01-03   8.4899  112.740  0.96509  16.185019  -0.4181   
15234   113514.SH  2020-01-06   9.0958  112.020  0.96509   4.805769  -0.2361   
17990   113539.SH  2020-01-06  -1.6554  126.980  2.99136   7.331114  -0.8035   
17991   113539.SH  2020-01-07  -1.6201  127.470  2.99136   8.487444  -0.8751   
...           ...         ...      ...      ...      ...        ...      ...   
138504  113565.SH  2023-09-04  37.8800  123.271  2.24800   5.030000  -2.4700   
278831  128033.SZ  2023-09-05  18.5100  119.712  2.97200   3.860000 -37.0000   
126994  113526.SH  2023-09-05  32.7600  129.742  1.92700   5.960000 -12.6700   
109332  113030.SH  2023-09-05  45.7300  119.287  2.94500   1.920000  -1.4900   
138505  113565.SH  2023-09-05  38.9000  122.754  2.24800   3.470000  -2.3000   

              yl        dl  ia

In [11]:
"""保存到EXCEL得时候重命名，便于阅读"""
column_mapping = {'date' : '日期',
                  'bond_code' : '可转债代码',
                  'dp' : '价格（收盘价全价）',
                  'cpr' : '转股溢价率',
                  'dl' : '双低',
                  'sn' : '正股名称',
                  'bl' : '债券余额（亿）',
                  'yl' : '剩余期限（年）',
                  'ytm' : '到期收益率', 
                  'trt' : '换手率', 
                  'low_price_midpoint' : '低价中点', 
                  'low_price_level' : '低价程度'}

high_ytm_5.rename(columns=column_mapping,inplace=True)
small_market_value_double_low_5.rename(columns=column_mapping,inplace=True)
double_low_5.rename(columns=column_mapping,inplace=True)
range_double_low_3.rename(columns=column_mapping,inplace=True)
low_price_small_market_value_5.rename(columns=column_mapping,inplace=True)
low_price_5.rename(columns=column_mapping,inplace=True)
double_low_pd_5.rename(columns=column_mapping,inplace=True)

time2 = time.time()

# 输出EXCEL
high_ytm_5.to_excel('pre_data_for_backtest\\高ytm（全量数据DTBS，用于回测）.xlsx',index=False)
small_market_value_double_low_5.to_excel('pre_data_for_backtest\\小市值3双低5（全量数据DTBS，用于回测）.xlsx',index=False)
double_low_5.to_excel('pre_data_for_backtest\\双低5（全量数据DTBS，用于回测）.xlsx',index=False)
range_double_low_3.to_excel('pre_data_for_backtest\\区间双低3（全量数据DTBS，用于回测）.xlsx',index=False)
low_price_small_market_value_5.to_excel('pre_data_for_backtest\\低价小市值5（全量数据DTBS，用于回测）.xlsx',index=False)
low_price_5.to_excel('pre_data_for_backtest\\低价5（全量数据DTBS，用于回测）.xlsx',index=False)

double_low_pd_5.to_excel('pre_data_for_backtest\\双低+平底5（全量数据DTBS，用于回测）.xlsx',index=False)

time3 = time.time()
print(f"运行时间：{time2-time1:.2f}秒, 存储时间：{time3-time2:.2f}秒, 总时间：{time3-time1:.2f}秒")

运行时间：4.75秒, 存储时间：22.36秒, 总时间：27.11秒
